In [1]:
import pandas as pd
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.vins import load_vins

pd.set_option('display.max_columns', 100)

vins = await load_vins(
  filter_comment='2024 Ford Ranger',
  include_openpilot=True,
  skip_reasons=set(),
)

# pre-load asbuilt
process_map(AsBuiltData.from_vin, vins, desc='Loading AsBuilt Data', chunksize=100)

vins

Loaded AsBuilt data for 4 VINs
Loaded 4 VINs (filter_comment='2024 Ford Ranger', include_openpilot=True, skipped=0, missing_asbuilt=0)


Loading AsBuilt Data:   0%|          | 0/4 [00:00<?, ?it/s]

['1FTER4KH8RLE02981',
 '1FTER4HH8RLE06231',
 '1FTER4GH7RLE07548',
 '8AFBR01L4RJ379379']

In [2]:
from panda.python.uds import DATA_IDENTIFIER_TYPE
from notebooks.ford.ecu import FordEcu, FordPart
from notebooks.ford.settings import VEHICLE_SETTINGS

ecus = {
  'PSCM': (FordEcu.PowerSteeringControlModule),
  'APIM (SYNC 4)': (FordEcu.AccessoryProtocolInterfaceModule, FordPart.APIM_SYNC4),
  'IPMA (Q4)': (FordEcu.ImageProcessingModuleA, FordPart.IPMA_Q4),
  'ABS': (FordEcu.AntiLockBrakeSystem),
  'C-CM': (FordEcu.CruiseControlModule),
}

abds: dict[str, AsBuiltData] = {vin: AsBuiltData.from_vin(vin) for vin in vins}


with pd.ExcelWriter('ford_ranger.xlsx', engine='xlsxwriter') as writer:
  for ecu_name, ecu in ecus.items():
    settings = [s for s in VEHICLE_SETTINGS if s.ecu == ecu]
    columns = [
      'VIN',
      *(s.comment for s in settings),
    ]
    rows = []

    for vin, abd in abds.items():
      rows.append([vin, *(abd.get_setting_value(s) for s in settings)])

    result = pd.DataFrame(rows, columns=columns).transpose()
    result.to_excel(writer, sheet_name=ecu_name)


  columns = [
    'VIN',
  ]
  for ecu_name in ecus.keys():
    columns.extend((
      f'{ecu_name} Part Number',
      f'{ecu_name} Software Version',
    ))

  rows = []
  for vin, abd in abds.items():
    row = [vin]
    for ecu in ecus.values():
      if not abd.is_present(ecu):
        row.extend(['not present', ''])
        continue

      ecu = abd.get_ecu(ecu)
      pn = str(abd.get_identifier(ecu, 0xF111))
      sw = str(abd.get_identifier(ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER))
      row.extend([pn, sw])
    rows.append(row)

  result = pd.DataFrame(rows, columns=columns).transpose()
  result.to_excel(writer, sheet_name='ECU Part Numbers')